In [19]:
import pandas as pd
import numpy as np
import send_to_racing
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
import time
from keras import regularizers
from keras.callbacks import EarlyStopping

In [20]:
bbox_data = np.array(send_to_racing.bbox)
#cleaning of bbox_data(removing class and probability since they little to no value in depth)
bbox_data = bbox_data[:432,3:5]
print(bbox_data)
print(bbox_data.shape)

[[0.015625   0.05277778]
 [0.015625   0.05416667]
 [0.02109375 0.07361111]
 [0.01875    0.0625    ]
 [0.05703125 0.16249999]
 [0.0609375  0.16805555]
 [0.025      0.08333334]
 [0.02578125 0.08611111]
 [0.03671875 0.10833333]
 [0.03515625 0.1       ]
 [0.01796875 0.05694444]
 [0.01796875 0.05416667]
 [0.01953125 0.07083333]
 [0.02109375 0.0625    ]
 [0.06171875 0.16111112]
 [0.05625    0.17083333]
 [0.02890625 0.09027778]
 [0.02890625 0.08611111]
 [0.0203125  0.07361111]
 [0.01796875 0.05555556]
 [0.01796875 0.05555556]
 [0.02109375 0.0625    ]
 [0.0625     0.16111112]
 [0.05703125 0.16944444]
 [0.028125   0.08888889]
 [0.028125   0.08611111]
 [0.0203125  0.07083333]
 [0.01875    0.05694444]
 [0.0171875  0.05555556]
 [0.02109375 0.06527778]
 [0.05859375 0.17222223]
 [0.02890625 0.08888889]
 [0.03828125 0.11527777]
 [0.028125   0.0875    ]
 [0.04140625 0.11527777]
 [0.01484375 0.04583333]
 [0.01796875 0.05694444]
 [0.015625   0.04583333]
 [0.02109375 0.07083333]
 [0.025      0.07916667]


In [21]:
depth = np.array(send_to_racing.depth_values)
depth = depth.reshape(441,1)
print(bbox_data.shape)

(432, 2)


In [22]:
model = Sequential([
    Dense(8, input_shape = (2, ), activation= 'relu'),
    Dense(40, activation='relu', kernel_regularizer=regularizers.L2(0.01)),
    Dropout(0),
    Dense(56, activation='relu'),
    Dropout(0.2),
    Dense(1, activation = 'relu')
])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.70,
                                patience=15, min_lr=0.000001)

In [23]:
model.compile(Adam(learning_rate=.01), loss='mse', metrics=['mse','F1Score'])
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model.fit(bbox_data, depth, validation_split=0.40, batch_size=10, epochs=700, shuffle = True, verbose=1, callbacks=[reduce_lr])

Epoch 1/700
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - F1Score: 1.6101 - loss: 22.4493 - mse: 22.3514 - val_F1Score: 1.6703 - val_loss: 6.9560 - val_mse: 7.1932 - learning_rate: 0.0100
Epoch 2/700
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - F1Score: 1.6470 - loss: 5.1512 - mse: 5.1129 - val_F1Score: 1.6703 - val_loss: 6.9016 - val_mse: 7.1456 - learning_rate: 0.0100
Epoch 3/700
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - F1Score: 1.6328 - loss: 4.3070 - mse: 4.2784 - val_F1Score: 1.6703 - val_loss: 5.7588 - val_mse: 5.9237 - learning_rate: 0.0100
Epoch 4/700
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - F1Score: 1.6362 - loss: 4.4640 - mse: 4.4356 - val_F1Score: 1.6703 - val_loss: 5.2306 - val_mse: 5.3652 - learning_rate: 0.0100
Epoch 5/700
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 977us/step - F1Score: 1.6440 - loss: 4.5985 - mse: 4.5637 - val_F1Score: 1.6703 - val_loss: 3.6903 - val_mse: 3.7544 - learning_rate: 0.0100
Epoch 6/700
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 919us/step - F1Score: 1.6597 - loss: 3.3821 - mse

In [24]:
test_data = np.array(send_to_racing.bbox)[432:,3:5]
start_time = time.time()
predicting = model.predict(test_data)
end_time = time.time()
print(f"average time taken{(end_time-start_time)/len(test_data)}")
print(predicting)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
average time taken0.004331323835584853
[[4.549363 ]
 [4.5803947]
 [2.7350419]
 [3.649685 ]
 [4.9396544]
 [2.882823 ]
 [2.2812963]
 [2.545075 ]
 [4.716954 ]]
